# Hyperparameter Tuning with Keras Tuner

In [2]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import sys
import warnings

warnings.filterwarnings('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

sys.setrecursionlimit(100000)

The sys.setrecursionlimit function is used to increase the recursion limit, which helps prevent potential recursion errors when running complex models with deep nested functions or when using certain libraries like TensorFlow.

## Data preprocessing

In [ ]:
(x_all, y_all), _ = mnist.load_data()

x_all = x_all.reshape((x_all.shape[0], -1)).astype('float32') / 255.0

# Split into train+val and test (80/20)
x_temp, x_test, y_temp, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

# Split train+val into train and validation (75/25 of 80% = 60/20 overall)
x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)

In [5]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print (f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


## Defining the model with hyperparameters

Define a model-building function that uses the `HyperParameters` object to specify the number of units in a dense layer and the learning rate. This function returns a compiled Keras model that is ready for hyperparameter tuning.

In [6]:
def build_model(hp):
  model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
    Dense(10, activation='softmax')
  ])

  model.compile(
    optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

  return model

## Configuring the hyperparameter search

This exercise guides you through configuring Keras Tuner. You create a `RandomSearch` tuner, specifying the model-building function, the optimization objective, the number of trials, and the directory for storing results. The search space summary provides an overview of the hyperparameters being tuned. 

In [8]:
tuner = kt.RandomSearch(
  build_model,
  objective='val_accuracy',
  max_trials=10,
  executions_per_trial=2,
  directory='hyperparameters',
  project_name='intro_to_kt'
)

tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


## Running the hyperparameter search

Run the hyperparameter search using the `search` method of the tuner. You provide the training and validation data along with the number of epochs. After the search is complete, the results summary displays the best hyperparameter configurations found. 

In [9]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

tuner.results_summary()

Trial 10 Complete [00h 00m 25s]
val_accuracy: 0.9783499836921692

Best val_accuracy So Far: 0.9793500006198883
Total elapsed time: 00h 04m 13s
Results summary
Results in hyperparameters/intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 08 summary
Hyperparameters:
units: 320
learning_rate: 0.0007967394249203055
Score: 0.9793500006198883

Trial 04 summary
Hyperparameters:
units: 384
learning_rate: 0.0006384825574016378
Score: 0.979200005531311

Trial 01 summary
Hyperparameters:
units: 288
learning_rate: 0.0022039876620071563
Score: 0.9786999821662903

Trial 09 summary
Hyperparameters:
units: 256
learning_rate: 0.0018009968632617217
Score: 0.9783499836921692

Trial 06 summary
Hyperparameters:
units: 352
learning_rate: 0.00034060994466741175
Score: 0.9774500131607056

Trial 03 summary
Hyperparameters:
units: 192
learning_rate: 0.00029871190678941467
Score: 0.9737000167369843

Trial 00 summary
Hyperparameters:
units: 416
learning_rate: 0.0001631581802

## Analyzing and using the best hyperparameters

Retrieve the best hyperparameters found during the search and print their values. You then build a model with these optimized hyperparameters and train it on the full training data set. Finally, you evaluate the model’s performance on the test set to ensure that it performs well with the selected hyperparameters. 

In [12]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f""" 
The optimal number of units in the first dense layer is {best_hps.get('units')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""") 

model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

test_loss, test_acc = model.evaluate(x_val, y_val)
print(f'Test accuracy: {test_acc}')

 
The optimal number of units in the first dense layer is 320.
The optimal learning rate for the optimizer is 0.0007967394249203055.

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9268 - loss: 0.2523 - val_accuracy: 0.9607 - val_loss: 0.1368
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9687 - loss: 0.1070 - val_accuracy: 0.9694 - val_loss: 0.1008
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9797 - loss: 0.0690 - val_accuracy: 0.9723 - val_loss: 0.0849
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9850 - loss: 0.0497 - val_accuracy: 0.9737 - val_loss: 0.0854
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9899 - loss: 0.0350 - val_accuracy: 0.9766 - val_loss: 0.0779
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9918 - loss: 0.0265 - val_accuracy: 0.9762 - val_loss: 0.0865
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9939 - loss: 0.020

## Exercise 1

Learn how to set up Keras Tuner and prepare the environment for hyperparameter tuning. 

1. Install Keras Tuner.
2. Import necessary libraries.
3. Load and preprocess the MNIST data se

In [ ]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


## Exercise 2

Define a model-building function that uses hyperparameters to configure the model architecture. 

1. Define a model-building function that uses the `HyperParameters` object to specify the number of units in a dense layer and the learning rate. 
2. Compile the model with sparse categorical cross-entropy loss and Adam optimizer. 

In [14]:
def build_model(hp):
  model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
    Dense(10, activation='softmax')
  ])

  model.compile(
    optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )
  
  return model

## Exercise 3

Set up Keras Tuner to search for the best hyperparameter configuration. 

1. Create a `RandomSearch` tuner using the model-building function. 
2. Specify the optimization objective, number of trials, and directory for storing results.

In [15]:
tuner = kt.RandomSearch(
  build_model,
  objective='val_accuracy',
  max_trials=10,
  executions_per_trial=2,
  directory='hyperparameters',
  project_name='intro_to_kt'
)

tuner.search_space_summary()

Reloading Tuner from hyperparameters/intro_to_kt/tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


## Exercise 4

Run the hyperparameter search and dispaly the summary of the results.

1. Run the hyperparameter search using the `search` method of the tuner. 
2. Pass in the training data, validation data, and the number of epochs. 
3. Display a summary of the results. 

In [16]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 
tuner.results_summary()

Results summary
Results in hyperparameters/intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 08 summary
Hyperparameters:
units: 320
learning_rate: 0.0007967394249203055
Score: 0.9793500006198883

Trial 04 summary
Hyperparameters:
units: 384
learning_rate: 0.0006384825574016378
Score: 0.979200005531311

Trial 01 summary
Hyperparameters:
units: 288
learning_rate: 0.0022039876620071563
Score: 0.9786999821662903

Trial 09 summary
Hyperparameters:
units: 256
learning_rate: 0.0018009968632617217
Score: 0.9783499836921692

Trial 06 summary
Hyperparameters:
units: 352
learning_rate: 0.00034060994466741175
Score: 0.9774500131607056

Trial 03 summary
Hyperparameters:
units: 192
learning_rate: 0.00029871190678941467
Score: 0.9737000167369843

Trial 00 summary
Hyperparameters:
units: 416
learning_rate: 0.000163158180275917
Score: 0.9726000130176544

Trial 07 summary
Hyperparameters:
units: 288
learning_rate: 0.004116480206370245
Score: 0.9715499877929688

Tr

## Exercise 5

Retrieve the best hyperparameters from the search and build a model with these optimized values. 

1. Retrieve the best hyperparameters using the `get_best_hyperparameters` method. 
2. Build a model using the best hyperparameters. 
3. Train the model on the full training data set and evaluate its performance on the validation set.

In [17]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 

print(f""" 
The optimal number of units in the first dense layer is {best_hps.get('units')}. 
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 
""") 

model = tuner.hypermodel.build(best_hps) 
model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

val_loss, val_acc = model.evaluate(x_val, y_val) 
print(f'Validation accuracy: {val_acc}')

 
The optimal number of units in the first dense layer is 320. 
The optimal learning rate for the optimizer is 0.0007967394249203055. 

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9264 - loss: 0.2562 - val_accuracy: 0.9594 - val_loss: 0.1398
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9694 - loss: 0.1031 - val_accuracy: 0.9715 - val_loss: 0.0924
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9800 - loss: 0.0669 - val_accuracy: 0.9729 - val_loss: 0.0903
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9856 - loss: 0.0476 - val_accuracy: 0.9769 - val_loss: 0.0758
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9905 - loss: 0.0338 - val_accuracy: 0.9769 - val_loss: 0.0772
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9927 - loss: 0.0253 - val_accuracy: 0.9762 - val_loss: 0.0860
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9943 - loss: 0.0